# Task 2: Data Integration for EV Charging Station Analysis

In this task, we focused on integrating diverse datasets to support the throughput forecasting of electric vehicle charging stations. The integrated datasets include:

- **PUN Data**: Sourced from Piattaforma Unica Nazionale, providing valuable insights into energy demand and competition.
- **Electric Vehicles Diffusion Data**: Offering regional data on the penetration and usage of electric vehicles to understand market demand.
- **Amenities Data**: Information about the presence of amenities such as restaurants, offices, and services in the vicinity of our chargers, collected to assess potential customer attraction points.

This integrated approach aims to enhance the analysis and prioritization of high-potential locations for EV charging stations.


In [ ]:
# importing libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import ast
import folium
import osmnx as ox
import geopandas as gpd
from shapely import wkt
import plotly.express as px

In [ ]:
cdr = pd.read_csv('RIKY/data/cdr_preprocessed.csv')
pdr = pd.read_csv('RIKY/data/pdr_processed.csv')

## INTEGRATION WITH AMENITIES

In [ ]:
# CODICE COMPLETO PER AMENITIES NEL RAGGIO DI 800M

# Extract unique latitude and longitude pairs and Convert to a list of tuples
unique_coords = pdr[['station_coordinates_lat', 'station_coordinates_lon']].drop_duplicates()
unique_coords_list = list(unique_coords.itertuples(index=False, name=None))

# Create an empty DataFrame to store all POI data
all_pois = pd.DataFrame()

# Function to extract POIs around each charging point
def extract_pois(lat, lon, radius=800):
    # Define the tags for relevant amenities
    tags = {
        'amenity': [
            'restaurant', 'cafe', 'fast_food', 'pub', 'bar', 'food_court',  # Food & dining
            'gym',  # Fitness centers
            'supermarket', 'grocery',  # Shopping & groceries
            'college', 'research_institute', 'university',  # Educational facilities
            'office', 'coworking_space',  # Workplaces
            'court_house', 'post_office',  # Public services
            'cinema', 'conference_centre', 'events_venue', 'exhibition_centre', 'theatre'  # Arts and entertainment
        ],
        'shop': ['mall', 'retail', 'department_store'],  # Shopping centers
    }
    
    try:
        # Download POIs within the radius using osmnx
        poi_data = ox.geometries_from_point((lat, lon), tags=tags, dist=radius)
        
        # Filter to keep only points and exclude polygons or multipolygons
        poi_data = poi_data[poi_data.geometry.type == 'Point']
        
        # If no points are found, return an empty DataFrame
        if poi_data.empty:
            return pd.DataFrame()

        # Convert to DataFrame and add latitude and longitude columns from geometry
        poi_df = pd.DataFrame(poi_data)
        poi_df['latitude'] = poi_data.geometry.y
        poi_df['longitude'] = poi_data.geometry.x
        
        # Return the filtered POI DataFrame
        return poi_df
    
    except Exception as e:
        print(f"Error fetching POIs for location ({lat}, {lon}): {e}")
        return pd.DataFrame()  # Return an empty DataFrame if an error occurs

# Loop through the list of unique coordinates and extract POIs
for lat, lon in unique_coords_list:
    pois = extract_pois(lat, lon)
    all_pois = pd.concat([all_pois, pois], ignore_index=True)  # Append to the main DataFrame

# Save the combined POI data to a CSV file
output_file_path = '800m_pois.csv'
all_pois.to_csv(output_file_path, index=False)
print(f"POI data has been saved to {output_file_path}")

In [ ]:
FILE_PATH = 'PATH TO YOUR/800m_pois.csv'
extracted_df = pd.read_csv('FILE_PATH')
extracted_df = extracted_df[['amenity', 'geometry', 'name']]
extracted_df.to_csv(FILE_PATH, index=False)

In [ ]:
# Code to plot amenities counter on your df

'''#PLOT AMENITIES COUNTER 

HERE YOU HAVE TO CHOSE THE FILE TO PLOT

# Define colors inspired by the Free To X logo
colors = ['#0071BC', '#FFD100', '#4D4D4F']  # Blue, yellow, gray

# Plot
plt.figure(figsize=(12, 6))
bars = plt.bar(df['amenity'], df['count'], color=colors[0])  # Using blue as the primary color

# Customize each bar color based on index for variety (using a repeating color pattern)
for i, bar in enumerate(bars):
    bar.set_color(colors[i % len(colors)])

# Title and labels
plt.title('Count of Amenities Near Charging Stations', fontsize=16, weight='bold')
plt.xlabel('Amenity Type', fontsize=14)
plt.ylabel('Count', fontsize=14)

# Rotate x-axis ticks for readability
plt.xticks(rotation=45, ha='right', fontsize=12)

# Customize y-axis ticks and grid
plt.yticks(fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Display the plot
plt.tight_layout()
plt.show()
'''

In [ ]:
# Code to plot amenities on your df in open street map style

# Convert the 'geometry' column from WKT format to actual geometry using shapely
extracted_df['geometry'] = extracted_df['geometry'].apply(wkt.loads)

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(extracted_df, geometry='geometry')

# Filter only POINT geometries for mapping, as polygons have multiple coordinates
point_data = gdf[gdf['geometry'].type == 'Point'].copy()

# Extract latitude and longitude from the POINT geometry
point_data['latitude'] = point_data['geometry'].y
point_data['longitude'] = point_data['geometry'].x

# Plot using plotly express
fig = px.scatter_mapbox(
    point_data,
    lat='latitude',
    lon='longitude',
    color='amenity',
    hover_name='amenity',
    zoom=6,
    mapbox_style='open-street-map',
    title='Visualizzazione dei Punti di Interesse in Italia'
)

# Show the map
fig.show()